In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
np.random.seed(2018)

trn = pd.read_csv('/home/jeongchanwoo/workspace/git/study/Kaggle_data/santander-product-recommendation/input/train_ver2.csv')
tst = pd.read_csv('/home/jeongchanwoo/workspace/git/study/Kaggle_data/santander-product-recommendation/input/test_ver2.csv')

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
## 데이터 전처리 ##

# 제품 변수
prods = trn.columns[24:].tolist()

# 제품 변수 결측값 0으로 대체
trn[prods] = trn[prods].fillna(0).astype(np.int8)

In [8]:
#24개 제품 중 하나도 보유하지 않은 고객 제거
no_product = trn[prods].sum(axis=1) ==0
trn = trn[~no_product]

In [10]:
# 훈련 데이터 테스트 데이터 통합 테스트 데이터에 없는 제품 변수 0으로 채움
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis = 0)

In [11]:
# 학습 변수 list
features = []


In [21]:
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp','canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']

In [22]:
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel = -99)
features += categorical_cols

In [37]:
df['renta'].unique()

array([87218.1, 35548.74, 122179.11000000002, ..., '  139164.12',
       '  100647.45', '   72765.27'], dtype=object)

In [38]:
# 수치형 변수의 특이값, 결측값 -99로 대체하고 정수형 변환
df['age'].replace(' NA', -99, inplace = True)
df['age']  = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace = True)
df['antiguedad']  = df['antiguedad'].astype(np.int8)

df['renta'].replace('      Na', -99, inplace = True)
df['renta'].fillna(-99, inplace=True)
df['renta']  = df['renta'].astype(np.int8)

df['indrel_1mes'].replace(' P', 5, inplace = True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes']  = df['indrel_1mes'].astype(np.int8)

ValueError: invalid literal for int() with base 10: '  326124.90'